In [1]:
import pandas as pd
from neo4j_graphrag.llm import LLMResponse, OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings

In [2]:
#pokemon = pd.read_csv("../data/pokemonDB_dataset.csv")

pokemon = pd.read_parquet("../data/pokemonDB_dataset.parquet")

In [3]:
api_key = "ollama"

llm = OpenAILLM(
    base_url="http://localhost:11434/v1",
    model_name="llama3",
    api_key=api_key,
)

embeder = OpenAIEmbeddings(
    base_url="http://localhost:11434/v1",
    api_key=api_key,
    model="snowflake-arctic-embed:latest",
)

def generate_description(question:str) -> str:
    res: LLMResponse = llm.invoke(question)
    return res.content

def generate_embeddings(text:str):
    embeddings = embeder.embed_query(text)
    return embeddings

# Generate a description for each Pokemon in the dataset
def generate_pokemon_descriptions(pokemon_df):
    descriptions = []
    for _, row in pokemon_df.iterrows():
        
        question = f"Create a short summary about {row['Pokemon']}"
        description = generate_description(question)

        descriptions.append(description)
    return descriptions

def generate_pokemon_description_embeddings(pokemon_df):
    embeddings = []
    for _, row in pokemon_df.iterrows():
        embedding = generate_embeddings(row['Description'])
        embeddings.append(embedding)
    return embeddings


#pokemon['Description'] = generate_pokemon_descriptions(pokemon)
pokemon['vectorProperty'] = generate_pokemon_description_embeddings(pokemon)

In [4]:
pokemon.head(2)

,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,...,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max,Description,vectorProperty
0,Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.2 m (7′03″),135.5 kg (298.7 lbs),"1. Snow Warning, Soundproof (hidden ability)","1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),173,...,170,311,85,157,295,60,112,240,Abomasnow!,"[0.038857594, -0.058271974, -0.007995857, 0.00..."
1,Mega Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.7 m (8′10″),185.0 kg (407.9 lbs),1. Snow Warning,"1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),208,...,242,399,105,193,339,30,58,174,"Mega Abomasnow is a powerful, Snow/Ice-type Po...","[-0.07787469, -0.015716149, -0.059320956, 0.01..."


In [5]:
pokemon.to_parquet("../data/pokemonDB_dataset.parquet", index=False)